<a href="https://colab.research.google.com/github/tomer9080/DL-Speech-exercises/blob/main/amdf_lpc_046747_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pitch estimation using AMDF, Linear Predictive Coding, LPC Spectrum Processing

In this exercise, you will deploy an end-to-end pipeline, that starts from recording an utterance. The next step is to analyze the acoustical properties of the recorded waveform, such as pitch and formant extraction and classification of voiced/unvoiced segments. To complete the task, you will implement the algorithm to find the LPC Coefficients, then process the LPC Spectrum in order to examine the effect of warping the LPC spectrum.

> You might use Google-Colab to complete the exercise, or use a local setup. If you choose to use local setup, make sure all required packages are installed.

## Report Guidelines for Plots and Speech Signal Figures

Follow these guidelines to ensure your report is clear, professional, and easy to evaluate:

### 1. Figures and Captions
- Every figure must have a **descriptive caption**.
- Ensure figures are readable (clear fonts, non-pixelated, proper line thickness).

### 2. Axes and Labels
- Always label axes with **names and units** (e.g., Time (s), Frequency (Hz)).
- Use consistent axis limits and scales when comparing signals.

### 3. Interpretation
- Do not just show plots—**explain** what they mean.
- Highlight key observations (peaks, trends, transitions).
- Compare your results with ground truth or theory when relevant.



## 🐍 Import python libraries

In [ ]:
import ast
import scipy
import librosa
import datasets
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

## 🎤 Loading the utterance

Load the file attached in the moodle. Note that are 2 files: a `.wav` file and a `.phn` file.

The sound file contains the utterance:

> *She had your dark suit in greasy wash water all year.*

The `.phn` file contains the mapping between samples and the phone said in the `.wav` file.

Load the sound file using the code below and play it.

In [ ]:
## Do not remove
import IPython.display as ipd

# Load the two recordings
y1, sr1 = librosa.load('sa1.wav', sr=16000)

# Play the first recording
print("Playing utterance:")
display(ipd.Audio(y1, rate=sr1))

Plot your recording signals using the code below:

In [ ]:
## Do not remove
plt.figure(figsize=(12, 6))

plt.subplot(1, 1, 1)
plt.plot(np.arange(len(y1)) / sr1, y1)
plt.title('Recording 1')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)

plt.tight_layout()
plt.show()

Now, run the below code to get a list that resembles the sequence of phones in the utterance.

Each entry in the list has a `phone` field, `start` field and `end` field. Both `start, end` units are samples.

In [ ]:
## Do not remove
with open("sa1.phn", "r") as file:
    lines = file.readlines()

phones = []
for line in lines:
    start, end, phone = line.strip().split()
    phones.append({"start": int(start), "end": int(end), "phone": phone})

print(f"{phones=}")

## 1. Pitch Estimation using the Average Magnitude Difference Function (AMDF)

In this section, you will learn what the Average Magnitude Difference Function (AMDF) is and how it can be used to estimate the pitch (fundamental frequency) of a speech signal.

The AMDF is computed over short-time frames and is defined as:

$$
D(\tau) = \frac{1}{N} \sum_{n=0}^{N-1} |x(n) - x(n+\tau)|
$$

Here, $x(n)$ represents the speech samples, $N$ is the analysis frame length, and $\tau$ is the time lag.

For a periodic signal with period $T_0$, the AMDF typically exhibits a clear minimum when $\tau = T_0$. Therefore, the pitch period can be estimated by selecting the lag value that minimizes the AMDF:

$$
T_0 = \arg\min_{\tau} D(\tau), \quad \tau \in [\tau_{\min}, \tau_{\max}]
$$

The values $\tau_{\min}$ and $\tau_{\max}$ should be chosen according to the expected pitch range.

In this task, we limit the pitch search range to frequencies between **50 Hz and 400 Hz**, which is a suitable range for typical speech signals. When implementing the method, this frequency range must be converted into corresponding lag values based on the sampling rate.

> **Note**: When choosing an interval of speech to evaluate the pitch frequency on, make sure you include at least 5 pitch cycles.  

### 1.1 Vanilla AMDF

In this part, you will apply the pitch estimation method described in Section 1 to your recorded utterances.

Begin by displaying the waveform of each student's utterance. From each utterance, select **one periodic segment** (voiced speech) and **one non-periodic segment** (unvoiced or noise-like speech). You can choose the segments by using the `phones` list provided earlier.
In the report, clearly specify:
- which segments you selected,
- the sample indices (boundaries) of the segments you will work on.

For each selected segment, compute and plot the AMDF function $D(\tau)$ alongside the corresponding time-domain signal.

The AMDF is computed as:
$$
D(\tau) = \frac{1}{N} \sum_{n=0}^{N-1} |x(n) - x(n+\tau)|
$$

For a periodic segment, $D(\tau)$ is expected to show a strong minimum at the pitch period $T_0$. In your plots, mark the estimated pitch period $T_0$ by identifying:
$$
T_0 = \arg\min_{\tau} D(\tau)
$$

For the non-periodic segment, you should observe that $D(\tau)$ does **not** exhibit a clear minimum, reflecting the lack of periodic structure.

Be sure to clearly annotate all plots, indicating:
- the waveform segment,
- the AMDF curve $D(\tau)$,
- and, for periodic segments, the estimated pitch period $T_0$.


In [ ]:
## Your code here

### 1.2 Circular AMDF

In this part, you will apply a modified version of the AMDF method by using its circular form.  
Use the same periodic and non-periodic segments selected in Section 1.1.

For each selected segment, compute and plot the Circular AMDF (CAMDF) function $D_c(\tau)$ alongside the corresponding time-domain signal.

The CAMDF is defined as:
$$
D_c(\tau) = \frac{1}{N} \sum_{n=0}^{N-1} \big| x((n+\tau) \bmod N) - x(n) \big|
$$

This formulation treats the signal segment as if it were **circularly periodic** with length $N$.  
Under this circularity assumption, the CAMDF satisfies the symmetry property:
$$
D_c(\tau) = D_c(N - \tau),
$$
which means that only the range $\tau \in [0, \frac{N}{2}]$ needs to be evaluated.

**Important considerations:**

- The symmetry property holds because of the *circular indexing*, not because the speech signal itself is symmetric.  
- Circular indexing may introduce artificial structure if the endpoints of the segment do not match.  
  Therefore, choose segment lengths that contain at least **3 pitch periods** to avoid misleading minima.
- Even for non-periodic segments, CAMDF may produce weak or spurious minima due to circular wrap-around.  
  You should interpret such minima cautiously.

For periodic segments, $D_c(\tau)$ is expected to exhibit a clear minimum at the pitch period $T_0$. Mark this point on your plots by identifying:
$$
T_0 = \arg\min_{\tau} D_c(\tau)
$$

For non-periodic segments, the CAMDF should not show a stable or prominent minimum, although small artificial minima may appear because of the circular structure.

In your report, include for each segment:
- the waveform segment,
- the CAMDF curve $D_c(\tau)$ over $\tau \in [0, \frac{N}{2}]$,
- and, for periodic segments, the estimated pitch period $T_0$.


In [ ]:
## Your code here

### 1.3 Comparison with Built-In Python Pitch Estimation

In this part, use the `librosa.yin` function to compute the pitch frequency for the periodic segments selected earlier. Compare the pitch values obtained from the built-in YIN algorithm with the estimates you obtained using the AMDF (Section 1.1) and Circular AMDF (Section 1.2).

In your report, clearly list the pitch estimates from all three methods:
- AMDF (Section 1.1)
- Circular AMDF (Section 1.2)
- `librosa.yin`

> **Note**: To retrieve the pitch value estimated from the YIN algorithm, apply `np.median` on `librosa.yin` result.

You should use this comparison as a sanity check to the correctness of your AMDF estimations.


In [ ]:
## Your code here

## 2. LPC spectrum and Formants Extraction

In this section, you will use the LPC spectrum to extract formants from speech utterances.

You will work on a dataset called *Hillenbrand*, which contains vowels utterances of children and adults (male and female). More specifically, the dataset contains words of the form */h-d/* where "-" is replaced with a different vowel.

Using this dataset, you will extract the formants for each vowel per each group (boys, girls, men, women) using the LPC spectrum and plot it on a scatter plot.

You will compare your extracted formants with the manually annotated formants in the dataset.

Load the dataset using the code provided below.

In [ ]:
## Do not remove
import ast
import datasets

ds = datasets.load_dataset("MLSpeech/hillenbrand_vowels", split="train")

def cast_all(example):
    for key in ["audio", "formant_1", "formant_2", "formant_3", "formant_4"]:
        if key in example:
            example[key] = np.array(ast.literal_eval(example[key]))
    return example

ds = ds.map(cast_all)

# sample keys
print(f"{ds[0].keys()}")

### 2.1 The LPC Spectrum

In order to extract the speech formants, we will use the LPC spectrum. As seen in class, the LPC spectrum yields a smoother spectral representation than the standard FFT spectrum. This smoothness allows us to better observe the formants and the pitch frequency of the speech signal.

In the code below, implement the function `get_lpc_spectrum`.

The function should:

- Take a speech signal `x` and a filter `order` as input.
- Compute LPC coefficients using `librosa.lpc`.
- Compute the LPC spectrum using `scipy.signal.freqz`.
- Return both the frequency vector and the LPC spectrum magnitude (in dB).

> **Reminder**: $\mathrm{dB}(x) = 20\log_{10}(|x|)$. It is a good practice to add a small quantity, $\epsilon=10^{-12}$ to the $\log$ argument for numerical stability.

In [ ]:
from scipy.signal import freqz

def get_lpc_spectrum(x, sr, order):
    """
    Computes the LPC spectrum of a speech signal.

    Parameters
    ----------
    x : np.ndarray
        Input speech signal.
    sr : int
        Sampling rate of the signal.
    order : int
        LPC filter order.

    Returns
    -------
    freqs : np.ndarray
        Frequency vector (Hz).
    lpc_spectrum_db : np.ndarray
        LPC spectrum magnitude in dB.
    """
    pass


* Select a single sample from the Hillenbrand dataset and plot its LPC spectrum. Include the plot in your report. Then, try to estimate the formants approximately by visually inspecting the LPC spectrum. Add your estimations to the report.

> **Note:** When computing the LPC spectrum of a vowel, use the steady-state portion at the center of the vowel—typically a 40 ms segment from the middle of the recording.


In [ ]:
## Your code here

### 2.2 Extracting Formants

Once we have the LPC spectrum, we can estimate the formants by finding the **peaks** in the spectrum. Formants typically correspond to the **local maxima** of the LPC spectrum.  

In Python, you can use `scipy.signal.find_peaks` to detect these maxima. You may want to keep only the first few formants (F0, F1, F2, F3) which are the most relevant for vowel analysis.

Implement the function `extract_formants` as described below.

The function should:

* Take the LPC spectrum and corresponding frequency vector as input.
* Find the peaks in the LPC spectrum using `find_peaks`.
* Return the first 4 formants (F0, F1, F2, F3) in Hz.

In [ ]:
import numpy as np
from scipy.signal import find_peaks

def extract_formants(freqs, lpc_spectrum):
    """
    Extract the first four formant frequencies (F0-F3) from an LPC spectrum.

    Args:
        freqs (np.ndarray): Frequency axis (Hz).
        lpc_spectrum (np.ndarray): LPC magnitude spectrum.

    Returns:
        list: [F0, F1, F2, F3] in Hz (if not enough peaks are found - default is -1).
    """
    pass

Now use the function `extract_formants` to compare the formants resulted from the function to the ones you approximated in section 2.1.

In [ ]:
## Your code here

### 2.3 Extract Formants from the Dataset

In this section, we use the Hillenbrand dataset, which contains vowel recordings categorized into four speaker groups: boys, girls, men, and women. The filenames indicate the speaker group (e.g., recordings from boys start with "b", from girls with "g", etc.).

Iterate through all `wav` files in the dataset and extract the formants for each file. Each extracted result should be organized by **group** and **vowel** (for example: *group: girl, vowel: ae*) to allow for later analysis.  
When computing the LPC spectrum, ensure that you use only the **vowel core**—a 40–50 ms segment taken from the center of each vowel recording.


In [ ]:
## Your code here

Now, compare the formants you estimated with the annotated results provided in the Hillenbrand dataset. For this task, focus only on **F1** and **F2**.

Each `.wav` file in the dataset has a corresponding formant array, where each element represents a 10ms frame of the speech signal. To compare your estimates with the ground truth:

1. Make sure to extract the correct time frames from the dataset formant arrays that correspond to the segment you analyzed.
2. Average the formants along the time axis to obtain a single value for **F1** and **F2**.

At this point, you should have **two sets of formants**: one from your LPC-based functions and one from the dataset annotations.

Next:

* Create a scatter plot of vowels in the **F1–F2 space**.
* Separate the plots by speaker groups: **boys, girls, women, men**.
* Present **two subfigures side by side**: one showing your estimated formants and one showing the dataset formants, per group (i.e, each sub-figure will contain scatter of vowels formants values per group, per each group. In total 4 groups of scatter plots on each sub figure). Make sure you average each vowel formants values over the group axis before plotting the scatter plots.
* Analyze and discuss the differences between your estimates and the annotated dataset values.


In [ ]:
## Your code here

# Good Luck!